In [1]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [2]:
import scipy.spatial as ss
from scipy.special import digamma
from math import log
import numpy.random as nr
import numpy as np
import random
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn import svm
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier
import math

In [3]:

def lcsi(X, y, **kwargs):
    n_samples, n_features = X.shape
    # index of selected features, initialized to be empty
    F = []
    # Objective function value for selected features
    J_CMI = []
    # Mutual information between feature and response
    MIfy = []
    # indicate whether the user specifies the number of features
    is_n_selected_features_specified = False
    # initialize the parameters
    if 'beta' in kwargs.keys():
        beta = kwargs['beta']
    if 'gamma' in kwargs.keys():
        gamma = kwargs['gamma']
    if 'n_selected_features' in kwargs.keys():
        n_selected_features = kwargs['n_selected_features']
        is_n_selected_features_specified = True

    # select the feature whose j_cmi is the largest
    # t1 stores I(f;y) for each feature f
    t1 = np.zeros(n_features)
    # t2 stores sum_j(I(fj;f)) for each feature f
    t2 = np.zeros(n_features)
    # t3 stores sum_j(I(fj;f|y)) for each feature f
    t3 = np.zeros(n_features)
    for i in range(n_features):
        f = X[:, i]
        t1[i] = midd(f, y)
    print(t1)
    # make sure that j_cmi is positive at the very beginning
    j_cmi = 1

    while True:
        if len(F) == 0:
            # select the feature whose mutual information is the largest
            idx = np.argmax(t1)
            F.append(idx)
            J_CMI.append(t1[idx])
            MIfy.append(t1[idx])
            f_select = X[:, idx]

        if is_n_selected_features_specified:
            if len(F) == n_selected_features:
                break
        else:
            if j_cmi < 0:
                break

        # we assign an extreme small value to j_cmi to ensure it is smaller than all possible values of j_cmi
        j_cmi = -1E30
        if 'function_name' in kwargs.keys():
            if kwargs['function_name'] == 'MRMR':
                beta = 1.0 / len(F)
            elif kwargs['function_name'] == 'JMI':
                beta = 1.0 / len(F)
                gamma = 1.0 / len(F)
        for i in range(n_features):
            if i not in F:
                f = X[:, i]
                t2[i] += midd(f_select, f)
                t3[i] += cmidd(f_select, f, y)
                # calculate j_cmi for feature i (not in F)
                t = t1[i] - beta*t2[i] + gamma*t3[i]
                # record the largest j_cmi and the corresponding feature index
                #print(t)
                if t > j_cmi:
                    j_cmi = t
                    idx = i
        F.append(idx)
        J_CMI.append(j_cmi)
        MIfy.append(t1[idx])
        f_select = X[:, idx]

    return np.array(F), np.array(J_CMI), np.array(MIfy)

In [4]:

# continuous estimators

def entropy(x, k=3, base=2):

    assert k <= len(x)-1, "Set k smaller than num. samples - 1"
    d = len(x[0])
    N = len(x)
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    tree = ss.cKDTree(x)
    nn = [tree.query(point, k+1, p=float('inf'))[0][k] for point in x]
    const = digamma(N)-digamma(k) + d*log(2)
    return (const + d*np.mean(map(log, nn)))/log(base)

def mi(x, y, k=3, base=2):

    assert len(x) == len(y), "Lists should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    y = [list(p + intens * nr.rand(len(y[0]))) for p in y]
    points = zip2(x, y)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = ss.cKDTree(points)
    dvec = [tree.query(point, k+1, p=float('inf'))[0][k] for point in points]
    a, b, c, d = avgdigamma(x, dvec), avgdigamma(y, dvec), digamma(k), digamma(len(x))
    return (-a-b+c+d)/log(base)

def cmi(x, y, z, k=3, base=2):
    """
    Mutual information of x and y, conditioned on z; x, y, z should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert len(x) == len(y), "Lists should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    intens = 1e-10  # small noise to break degeneracy, see doc.
    x = [list(p + intens * nr.rand(len(x[0]))) for p in x]
    y = [list(p + intens * nr.rand(len(y[0]))) for p in y]
    z = [list(p + intens * nr.rand(len(z[0]))) for p in z]
    points = zip2(x, y, z)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = ss.cKDTree(points)
    dvec = [tree.query(point, k+1, p=float('inf'))[0][k] for point in points]
    a, b, c, d = avgdigamma(zip2(x, z), dvec), avgdigamma(zip2(y, z), dvec), avgdigamma(z, dvec), digamma(k)
    return (-a-b+c+d)/log(base)

def kldiv(x, xp, k=3, base=2):
    """
    KL Divergence between p and q for x~p(x), xp~q(x); x, xp should be a list of vectors, e.g. x = [[1.3],[3.7],[5.1],[2.4]]
    if x is a one-dimensional scalar and we have four samples
    """

    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    assert k <= len(xp) - 1, "Set k smaller than num. samples - 1"
    assert len(x[0]) == len(xp[0]), "Two distributions must have same dim."
    d = len(x[0])
    n = len(x)
    m = len(xp)
    const = log(m) - log(n-1)
    tree = ss.cKDTree(x)
    treep = ss.cKDTree(xp)
    nn = [tree.query(point, k+1, p=float('inf'))[0][k] for point in x]
    nnp = [treep.query(point, k, p=float('inf'))[0][k-1] for point in x]
    return (const + d*np.mean(map(log, nnp))-d*np.mean(map(log, nn)))/log(base)

# Discrete estimators
def entropyd(sx, base=2):
    """
    Discrete entropy estimator given a list of samples which can be any hashable object
    """

    return entropyfromprobs(hist(sx), base=base)


def midd(x, y):
    """
    Discrete mutual information estimator given a list of samples which can be any hashable object
    """

    return -entropyd(list(zip(x, y)))+entropyd(x)+entropyd(y)


def cmidd(x, y, z):
    """
    Discrete mutual information estimator given a list of samples which can be any hashable object
    """

    return entropyd(list(zip(y, z)))+entropyd(list(zip(x, z)))-entropyd(list(zip(x, y, z)))-entropyd(z)


def hist(sx):
    # Histogram from list of samples
    d = dict()
    for s in sx:
        d[s] = d.get(s, 0) + 1
    return map(lambda z: float(z)/len(sx), d.values())


def entropyfromprobs(probs, base=2):
    # Turn a normalized list of probabilities of discrete outcomes into entropy (base 2)
    return -sum(map(elog, probs))/log(base)


def elog(x):
    # for entropy, 0 log 0 = 0. but we get an error for putting log 0
    if x <= 0. or x >= 1.:
        return 0
    else:
        return x*log(x)


# Mixed estimators
def micd(x, y, k=3, base=2, warning=True):
    """ If x is continuous and y is discrete, compute mutual information
    """

    overallentropy = entropy(x, k, base)
    n = len(y)
    word_dict = dict()
    for sample in y:
        word_dict[sample] = word_dict.get(sample, 0) + 1./n
    yvals = list(set(word_dict.keys()))

    mi = overallentropy
    for yval in yvals:
        xgiveny = [x[i] for i in range(n) if y[i] == yval]
        if k <= len(xgiveny) - 1:
            mi -= word_dict[yval]*entropy(xgiveny, k, base)
        else:
            if warning:
                print("Warning, after conditioning, on y={0} insufficient data. Assuming maximal entropy in this case.".format(yval))
            mi -= word_dict[yval]*overallentropy
    return mi  # units already applied


# Utility functions
def vectorize(scalarlist):
    """
    Turn a list of scalars into a list of one-d vectors
    """

    return [(x,) for x in scalarlist]


def shuffle_test(measure, x, y, z=False, ns=200, ci=0.95, **kwargs):
    """
    Shuffle test
    Repeatedly shuffle the x-values and then estimate measure(x,y,[z]).
    Returns the mean and conf. interval ('ci=0.95' default) over 'ns' runs, 'measure' could me mi,cmi,
    e.g. Keyword arguments can be passed. Mutual information and CMI should have a mean near zero.
    """

    xp = x[:]   # A copy that we can shuffle
    outputs = []
    for i in range(ns):
        random.shuffle(xp)
        if z:
            outputs.append(measure(xp, y, z, **kwargs))
        else:
            outputs.append(measure(xp, y, **kwargs))
    outputs.sort()
    return np.mean(outputs), (outputs[int((1.-ci)/2*ns)], outputs[int((1.+ci)/2*ns)])


# Internal functions
def avgdigamma(points, dvec):
    # This part finds number of neighbors in some radius in the marginal space
    # returns expectation value of <psi(nx)>
    N = len(points)
    tree = ss.cKDTree(points)
    avg = 0.
    for i in range(N):
        dist = dvec[i]
        # subtlety, we don't include the boundary point,
        # but we are implicitly adding 1 to kraskov def bc center point is included
        num_points = len(tree.query_ball_point(points[i], dist-1e-15, p=float('inf')))
        avg += digamma(num_points)/N
    return avg


def zip2(*args):
    # zip2(x,y) takes the lists of vectors and makes it a list of vectors in a joint space
    # E.g. zip2([[1],[2],[3]],[[4],[5],[6]]) = [[1,4],[2,5],[3,6]]
    return [sum(sublist, []) for sublist in zip(*args)]


In [5]:
def mrmr(X, y, **kwargs):
  
    if 'n_selected_features' in kwargs.keys():
        n_selected_features = kwargs['n_selected_features']
        F, J_CMI, MIfy = lcsi(X, y, gamma=0, function_name='MRMR', n_selected_features=n_selected_features)
    else:
        F, J_CMI, MIfy = lcsi(X, y, gamma=0, function_name='MRMR')
    return F, J_CMI, MIfy

In [6]:
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# load data
dataset = 'CreditRisk.csv'
link = '/content/drive/MyDrive/FeatureSelection/dataset/' + dataset
df=pd.read_csv(link)
nRow, nCol = df.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,45,53) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_col

In [9]:
#n_samples = 11000;
#df = df.groupby('y', group_keys=False).apply(lambda x: x.sample(frac=n_samples/nRow))
df.shape

(855969, 73)

In [10]:
#df = df.dropna(axis='rows')
cat_columns = df.select_dtypes(['object']).columns
if not cat_columns.empty:
  df[cat_columns] = df[cat_columns].apply(lambda x: pd.factorize(x)[0])
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,y
0,1077501,1296599,5000,5000,4975.0,0,10.65,162.87,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1077430,1314167,2500,2500,2500.0,1,15.27,59.83,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1077175,1313524,2400,2400,2400.0,0,15.96,84.33,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1076863,1277178,10000,10000,10000.0,0,13.49,339.31,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1075358,1311748,3000,3000,3000.0,1,12.69,67.79,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [11]:
df = df.fillna(0)
X = df.drop(columns=['id','y']).to_numpy()

features_label = df.columns.difference(['y']).to_numpy()
y = df['y'].to_numpy()
n_samples, n_features = X.shape
del df


In [12]:
start_time=time.time()
# perform evaluation on classification task
num_fea = n_features    # number of selected features

idx,_,_ = mrmr(X, y, n_selected_features=num_fea)

# obtain the dataset on the selected features
features = X[:, idx[0:num_fea]]

#print(score)
print(idx)



[3.04331207e-01 6.92215797e-03 6.97866533e-03 1.38202162e-02
 6.87590447e-04 4.23805226e-02 7.95668608e-02 1.07327684e-02
 1.15614480e-02 1.54814986e-01 4.19112088e-04 9.12140102e-04
 2.39631336e-02 1.79219178e-03 4.33266762e-02 4.70368351e-07
 8.62265545e-02 2.20566815e-03 6.07507724e-02 1.98416922e-03
 7.78459955e-04 4.28436861e-03 1.05398079e-04 1.50096108e-03
 3.82038348e-03 4.07809590e-04 1.26554998e-03 5.01995360e-04
 3.69004669e-04 4.56514575e-02 2.93342388e-03 5.61544645e-04
 7.31664062e-03 1.02205291e-01 1.02335810e-01 2.63054965e-01
 2.60328075e-01 2.51487134e-01 2.02759016e-01 1.51661494e-02
 1.31353471e-01 1.24372970e-01 1.11122017e-01 1.17798578e-01
 8.93293292e-02 3.64880345e-02 1.06824613e-04 8.35105318e-04
 0.00000000e+00 4.15914821e-05 4.15914821e-05 4.15914821e-05
 4.15914821e-05 1.34629759e-05 4.74962764e-03 1.40885233e-01
 7.61926732e-04 1.12736250e-03 6.13972054e-04 9.34624888e-04
 1.22562953e-03 1.13079833e-03 1.09301246e-03 8.36341840e-04
 1.09530200e-03 1.236810

In [13]:
idx_top10 = idx[0:int(n_features/10)]
print("--------------------------------------")
print("index of top  10 : %s" %idx_top10)
#print("Top 10 feature: %s " %features_label[idx_top10])

--------------------------------------
index of top  10 : [ 0 48 15 49 52 40 50]


In [14]:
idx_top15 = idx[0:int(15*n_features/100)]
print("--------------------------------------")
print("index of top  15 : %s" %idx_top15)
#print("Top 15 feature: %s " %features_label[idx_top15])

--------------------------------------
index of top  15 : [ 0 48 15 49 52 40 50 51 53 41]


In [15]:
idx_top25 = idx[0:int(n_features/4)]
print("--------------------------------------")
print("index of top  25: %s" %idx_top25)
#print("Top 25 feature: %s " %features_label[idx_top25])

print("--- %s seconds ---" % (time.time() - start_time))

--------------------------------------
index of top  25: [ 0 48 15 49 52 40 50 51 53 41 58 68 46 39 44 69 56]
--- 8105.705377101898 seconds ---


In [16]:
X_selected = X[:, idx_top10]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)

# fit and predict
model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

y_predict
acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', accuracy_score(y_test,y_predict))
print('ROC AUC =', roc_auc_score(y_test, y_predict ))
print('F1 =', f1_score(y_test,y_predict))

df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'MRMR','name':[dataset], 'samples': [n_samples], 'top': '10','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)

Learning rate set to 0.167384
0:	learn: 0.9732146	total: 206ms	remaining: 3m 25s
1:	learn: 0.9739374	total: 357ms	remaining: 2m 58s
2:	learn: 0.9739374	total: 497ms	remaining: 2m 45s
3:	learn: 0.9739374	total: 637ms	remaining: 2m 38s
4:	learn: 0.9739374	total: 785ms	remaining: 2m 36s
5:	learn: 0.9739360	total: 929ms	remaining: 2m 33s
6:	learn: 0.9739360	total: 1.08s	remaining: 2m 32s
7:	learn: 0.9739360	total: 1.2s	remaining: 2m 28s
8:	learn: 0.9739360	total: 1.35s	remaining: 2m 29s
9:	learn: 0.9739360	total: 1.49s	remaining: 2m 27s
10:	learn: 0.9739360	total: 1.63s	remaining: 2m 26s
11:	learn: 0.9739374	total: 1.77s	remaining: 2m 25s
12:	learn: 0.9739374	total: 1.91s	remaining: 2m 25s
13:	learn: 0.9739374	total: 2.06s	remaining: 2m 24s
14:	learn: 0.9739374	total: 2.2s	remaining: 2m 24s
15:	learn: 0.9739374	total: 2.37s	remaining: 2m 25s
16:	learn: 0.9739374	total: 2.52s	remaining: 2m 25s
17:	learn: 0.9739374	total: 2.66s	remaining: 2m 25s
18:	learn: 0.9739374	total: 2.78s	remaining: 2

In [17]:
X_selected = X[:, idx_top15]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)

# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', acc)
print('ROC AUC =', auc)
print('F1 =', f1)

df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'MRMR','name':[dataset], 'samples': [n_samples], 'top': '15','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)


Learning rate set to 0.167384
0:	learn: 0.9739374	total: 164ms	remaining: 2m 43s
1:	learn: 0.9739374	total: 310ms	remaining: 2m 34s
2:	learn: 0.9739374	total: 462ms	remaining: 2m 33s
3:	learn: 0.9739374	total: 612ms	remaining: 2m 32s
4:	learn: 0.9739374	total: 760ms	remaining: 2m 31s
5:	learn: 0.9739374	total: 910ms	remaining: 2m 30s
6:	learn: 0.9738483	total: 1.06s	remaining: 2m 30s
7:	learn: 0.9738483	total: 1.19s	remaining: 2m 28s
8:	learn: 0.9738483	total: 1.35s	remaining: 2m 28s
9:	learn: 0.9738483	total: 1.49s	remaining: 2m 27s
10:	learn: 0.9739374	total: 1.64s	remaining: 2m 27s
11:	learn: 0.9739374	total: 1.76s	remaining: 2m 25s
12:	learn: 0.9739374	total: 1.91s	remaining: 2m 25s
13:	learn: 0.9739374	total: 2.04s	remaining: 2m 23s
14:	learn: 0.9739374	total: 2.2s	remaining: 2m 24s
15:	learn: 0.9739374	total: 2.33s	remaining: 2m 23s
16:	learn: 0.9739374	total: 2.48s	remaining: 2m 23s
17:	learn: 0.9739374	total: 2.63s	remaining: 2m 23s
18:	learn: 0.9739374	total: 2.77s	remaining: 

In [18]:
X_selected = X[:, idx_top25]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)

# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

y_predict
acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', accuracy_score(y_test,y_predict))
print('ROC AUC =', roc_auc_score(y_test, y_predict ))
print('F1 =', f1_score(y_test,y_predict))

df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'MRMR','name':[dataset], 'samples': [n_samples], 'top': '25','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)

Learning rate set to 0.167384
0:	learn: 0.9739374	total: 164ms	remaining: 2m 43s
1:	learn: 0.9739374	total: 321ms	remaining: 2m 39s
2:	learn: 0.9744675	total: 487ms	remaining: 2m 41s
3:	learn: 0.9742368	total: 647ms	remaining: 2m 41s
4:	learn: 0.9742456	total: 818ms	remaining: 2m 42s
5:	learn: 0.9744091	total: 967ms	remaining: 2m 40s
6:	learn: 0.9742937	total: 1.13s	remaining: 2m 40s
7:	learn: 0.9744339	total: 1.28s	remaining: 2m 38s
8:	learn: 0.9744602	total: 1.44s	remaining: 2m 38s
9:	learn: 0.9745113	total: 1.59s	remaining: 2m 37s
10:	learn: 0.9745113	total: 1.75s	remaining: 2m 37s
11:	learn: 0.9744690	total: 1.91s	remaining: 2m 37s
12:	learn: 0.9745011	total: 2.06s	remaining: 2m 36s
13:	learn: 0.9745055	total: 2.21s	remaining: 2m 35s
14:	learn: 0.9745040	total: 2.37s	remaining: 2m 35s
15:	learn: 0.9745040	total: 2.51s	remaining: 2m 34s
16:	learn: 0.9745040	total: 2.68s	remaining: 2m 35s
17:	learn: 0.9745113	total: 2.86s	remaining: 2m 35s
18:	learn: 0.9745040	total: 3.01s	remaining: